# Build a reduced ETC model from RECON3D

In [1]:
from pytfa.redgem.lumpgem import LumpGEM
from pytfa.io.json import load_json_model
from cobra import Reaction
from cobra.util import Zero

recon3d = load_json_model('./../data/GEM_Recon3_thermocurated_redHUMAN.json')
recon3d.solver.configuration.tolerances.feasibility = 1e-9
recon3d.solver.configuration.tolerances.optimality = 1e-9 

2024-05-01 11:07:28,780 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [2]:
# Define the core model as all reactions that connect the ETC intermediates (RedGEM, paper = D0 ) 
# Quinones, Cytochromes, and ATP synthase

ubiquinone_reqations = [r.id for r in recon3d.metabolites.q10_m.reactions if recon3d.metabolites.q10h2_m in r.metabolites]
ferrocytochrome_reactions = [r.id for r in recon3d.metabolites.focytC_m.reactions if recon3d.metabolites.ficytC_m in r.metabolites]

# Malate aspartate shuttle is not for muscle (liver and others)
#aspartate_malate_shuttle = ['MDH', 'ASPTA', 'ASPTAm', 'MDHm', 'ASPGLUm', 'MALOAtm']

core_reaction_ids = list(set(ubiquinone_reqations + ferrocytochrome_reactions + ['ATPS4mi']))
core_reactions = [r for r in recon3d.reactions if r.id in core_reaction_ids]

# Print a table with the core reactions
print('Core reactions')
print('--------------')
for r in core_reactions:
    print(r.id, r.reaction)



Core reactions
--------------
DHORD9 dhor_S_c + q10_m --> orot_c + q10h2_m
ETFQO etfrd_m + q10_m --> etfox_m + q10h2_m
L_LACDcm 2.0 ficytC_m + lac_L_c --> 2.0 focytC_m + 2.0 h_c + pyr_c
SULFOX 2.0 ficytC_m + h2o_c + so3_c --> 2.0 focytC_m + 2.0 h_c + so4_c
r0205 glyc3p_c + q10_m --> dhap_c + q10h2_m
r0310 pmtcoa_m + q10_m --> hdd2coa_m + q10h2_m
r0509 q10_m + succ_m --> fum_m + q10h2_m
r0541 glutcoa_m + h_m + q10_m --> b2coa_m + co2_m + q10h2_m
r0560 ibcoa_m + q10_m --> 2mp2coa_m + q10h2_m
r0603 2mbcoa_m + q10_m --> 2mb2coa_m + q10h2_m
r0655 ivcoa_m + q10_m --> 3mb2coa_m + q10h2_m
r1446 btcoa_m + q10_m --> b2coa_m + q10h2_m
r1447 ddcacoa_m + q10_m --> dd2coa_m + q10h2_m
r1448 occoa_m + q10_m --> HC01415_m + q10h2_m
r1449 q10_m + tdcoa_m --> HC01412_m + q10h2_m
r1450 hxcoa_m + q10_m --> hx2coa_m + q10h2_m
r1451 dcacoa_m + q10_m --> dc2coa_m + q10h2_m
FADH2ETC fadh2_m + q10_m --> fad_m + h_m + q10h2_m
HMR_3859 2.0 ficytC_m + lac_D_c --> 2.0 focytC_m + 2.0 h_c + pyr_c
HMR_6500 CE5021_c + 

In [3]:
# # Print aspartate malate shuttle reactions
# print('Aspartate malate shuttle reactions')
# print('----------------------------------')
# for r_id in aspartate_malate_shuttle:
#     r = recon3d.reactions.get_by_id(r_id)
#     print(r.id, r.reaction)
    

In [4]:
###########################################
## Prepare the model lumpGEM
###########################################


# Add dummy reactions for ATP hydrolysis
cyt_atp2adp = Reaction('cyt_atp2adp')
recon3d.add_reactions([ cyt_atp2adp,])
cyt_atp2adp.reaction = 'atp_c + h2o_c --> adp_c + pi_c + h_c'

# ADD Phosphate translocase reaction
PiC = Reaction('PiC')
recon3d.add_reactions([ PiC,])
PiC.reaction = 'pi_c + h_c -> pi_m + h_m'


# This converts cobra reactions to pytfa (adds binary variables/constrains)
recon3d.objective = Zero # Symbolic zero objective

recon3d.prepare()
recon3d.convert()



2024-05-01 11:08:32,640 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


2024-05-01 11:08:47,776 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-05-01 11:08:47,777 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-05-01 11:08:47,779 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:08:47,780 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:08:47,783 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-05-01 11:08:47,785 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-05-01 11:08:47,786 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:08:47,788 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:08:47,826 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-05-01 11:08:47,828 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

In [5]:
# For each reaction remove delta
EPSILON = 1e-6
for g in recon3d.delta_gstd:
    lb = g.variable.lb
    ub = g.variable.ub
    mid = (lb + ub) / 2
    g.variable.lb = mid - EPSILON
    g.variable.ub = mid + EPSILON

In [6]:
recon3d.reactions.get_by_id('cyt_atp2adp').bounds = (1, 200)
recon3d.reactions.get_by_id('PiC').bounds = (0, 400)

In [7]:
# Open bounds from -100 -> -200 and 100 to 200
for r in recon3d.reactions:
    if r.bounds[0] == -100:
        r.lower_bound = -200
    if r.bounds[1] == 100:
        r.upper_bound = 200 

In [8]:
# By default only allow secretions for lumping + defined medium
for rxn in recon3d.boundary:
    rxn.bounds = (0, 200)

# Allow for protons exchange and oxygen uptake
recon3d.reactions.get_by_id('EX_h_e').bounds = (-200.0, 200.0)
recon3d.reactions.get_by_id('EX_h2o_e').bounds = (-200.0, 200.0)
recon3d.reactions.get_by_id('EX_o2_e').bounds = (-200.0, 0.0)

# Mal/Asp shuttle open
recon3d.reactions.get_by_id('ASPGLUm').bounds = (-200, 200)

# Test the model (For lumpting close all carbon sources)
recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_glc_D_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_bhb_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_hdca_e').bounds = (-1, 100)

recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)
recon3d.reactions.get_by_id('ATPtm').bounds = (-200,200)

# These are some FA reaction that can cause weird cycling
recon3d.reactions.r0310.bounds = (0, 0)
recon3d.reactions.HMR_3121.bounds = (0, 0)

# Block nadph reactions (erros in beta ox)
for r in recon3d.metabolites.nadp_m.reactions:
    r.bounds = (0,0)
for r in recon3d.metabolites.nadp_c.reactions:
    r.bounds = (0,0)


recon3d.objective = recon3d.reactions.get_by_id('cyt_atp2adp')

sol = recon3d.optimize()


In [9]:
# Print non-zero fluxes that are not transport reactions
for r in recon3d.reactions:
    if abs(sol.fluxes[r.id]) > 1e-6  and recon3d.metabolites.get_by_id('h_i') in r.metabolites:
        print(r.id, sol.fluxes[r.id], r.reaction, r.check_mass_balance())


ATPS4mi 100.0 adp_m + 4.0 h_i + pi_m --> atp_m + h2o_m + 3.0 h_m {}
CYOR_u10mi 45.99999999999999 2.0 ficytC_m + 2.0 h_m + q10h2_m --> 2.0 focytC_m + 4.0 h_i + q10_m {'charge': -4.0, 'Fe': -2.0}
NADH2_u10mi 31.0 6.0 h_m + nadh_m + q10_m --> 4.0 h_i + nad_m + q10h2_m {}
CYOOm2i 22.999999999999996 4.0 focytC_m + 8.0 h_m + o2_m --> 4.0 ficytC_m + 2.0 h2o_m + 4.0 h_i {'charge': 8.0, 'Fe': 4.0}


In [10]:
sol = recon3d.optimize()

In [11]:
# Print non-zero fluxes that are not transport reactions
for r in recon3d.reactions:
    if abs(sol.fluxes[r.id]) > 1e-6 and recon3d.metabolites.pi_m in r.metabolites:
        print(r.id, sol.fluxes[r.id], r.reaction, r.check_mass_balance())

SUCOAS1m -8.0 coa_m + gtp_m + succ_m <=> gdp_m + pi_m + succoa_m {}
ATPS4mi 100.0 adp_m + 4.0 h_i + pi_m --> atp_m + h2o_m + 3.0 h_m {}
PiC 108.0 h_c + pi_c --> h_m + pi_m {}


In [12]:
# recon3d.reactions.get_by_id('ATPS4mi').bounds = (0,200)
# recon3d.objective = recon3d.reactions.get_by_id('EX_lac_L_e')
# sol = recon3d.optimize()
# print(sol)
# # Constraint max lactate secretion
# # recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (sol.objective_value, 100)

In [13]:
# Test the model (For lumpting close all carbon sources)
recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_glc_D_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_bhb_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_hdca_e').bounds = (0, 100)

In [14]:
# Generate a lump for each carbon source 
EPSILON = 1e-6

# LumpGEM parameters
params = {
    'core_subsystems': [],
    'extracellular_system': [],
    'timeout': 3600,  # max time in s
    "constraint_method": 'both',
    # Stuff we dont need for this purpose
    "small_metabolites": [],
    "cofactor_pairs": [],
    "inorganics": [],
    "max_lumps_per_BBB": 10  # Maximal number of alternatives
}

In [15]:
def find_lumps(carbon_source, n_carbon, model, params=params, 
                           core_reactions=[], method='min', output='EX_co2_e'):  
        # Lump the reactions 
        
        params['growth_rate'] = n_carbon-EPSILON
        params['biomass_rxns'] =  [output,]

        resp_model = model.copy()
    
        resp_model.reactions.get_by_id(carbon_source).bounds = (-1-EPSILON, -1+EPSILON)
        resp_model.objective = resp_model.reactions.get_by_id(output)
        resp_model.objective_direction = 'max'
        sol = resp_model.optimize()
        print("Max CO2 production: ", sol.objective_value)

        # Maximal ATP production 
        resp_model.reactions.get_by_id(output).bounds = (n_carbon-EPSILON, n_carbon+EPSILON)
        resp_model.objective = resp_model.reactions.get_by_id('cyt_atp2adp')
        resp_model.objective_direction = 'max'
        sol = resp_model.optimize()
        print("Max ATP production: ", sol.objective_value)

        # Constraint max ATP production
        resp_model.reactions.get_by_id('cyt_atp2adp').bounds = (sol.objective_value-EPSILON*n_carbon, 200)

        # Reset CO2 constraint
        resp_model.reactions.get_by_id(output).bounds = (0, 100)

        resp_model.objective = Zero
        sol = resp_model.optimize()

        print("Test feasability: ", sol.objective_value)
        
        #LumpGEM needs to take as list of core reaction id as input
        subnetwork_extraction = LumpGEM(resp_model, core_reactions, params, min_transport=True, bigM=200 )
        lumps = subnetwork_extraction.compute_lumps(force_solve=False, method=method)

        return lumps

In [16]:
# Find the lumps for each carbon source to CO2
lumps = {}

In [17]:
carbon_source = "EX_hdca_e"
n_carbon = 16

# Force ATPs to be active for the respiratory lumps 
recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, 
                                    core_reactions=core_reaction_ids, method='min')

2024-05-01 11:10:48,440 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  16.000016
Max ATP production:  106.00010599999995
Test feasability:  0.0
Timeout limit is 3600s


2024-05-01 11:13:18,585 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-05-01 11:13:33,865 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-05-01 11:13:33,866 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-05-01 11:13:33,868 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:13:33,870 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:13:33,873 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-05-01 11:13:33,875 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-05-01 11:13:33,877 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:13:33,879 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:13:33,923 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-05-01 11:13:33,924 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [07:19<00:00, 439.48s/it]


In [18]:
# Compute a lump for glycolysis (Direct respiration makes no sense ... )
carbon_source = 'EX_glc_D_e'
n_carbon = 2
output = 'EX_lac_L_e'

# No need
recon3d.reactions.get_by_id('ATPS4mi').bounds = (0,200)

# Add glycolysis lumps
lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d,
                                   core_reactions=core_reaction_ids, method='min', output=output)


2024-05-01 11:22:14,255 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  2.000001999999995
Max ATP production:  2.000001
Test feasability:  0.0
Timeout limit is 3600s


2024-05-01 11:24:56,117 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-05-01 11:25:10,795 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-05-01 11:25:10,796 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-05-01 11:25:10,797 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:25:10,800 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:25:10,803 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-05-01 11:25:10,805 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-05-01 11:25:10,807 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:25:10,809 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:25:10,853 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-05-01 11:25:10,855 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=lac_L_e: 100%|██████████| 1/1 [05:45<00:00, 345.61s/it]


In [19]:
carbon_source = "EX_lac_L_e"
n_carbon = 3

# Force ATPs to be active for the respiratory lumps 
recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, 
                                    core_reactions=core_reaction_ids, method='min')

2024-05-01 11:32:18,343 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  3.0000029999999924
Max ATP production:  15.000015000000015
Test feasability:  0.0
Timeout limit is 3600s


2024-05-01 11:34:54,794 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-05-01 11:35:09,819 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-05-01 11:35:09,820 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-05-01 11:35:09,821 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:35:09,823 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:35:09,826 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-05-01 11:35:09,830 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-05-01 11:35:09,833 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:35:09,834 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:35:09,858 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-05-01 11:35:09,860 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [05:31<00:00, 331.37s/it]


In [20]:
carbon_source = "EX_bhb_e"
n_carbon = 4

# Force ATPs to be active for the respiratory lumps 
recon3d.reactions.get_by_id('ATPS4mi').bounds = (1,200)

lumps[carbon_source] = find_lumps(carbon_source, n_carbon, recon3d, 
                                    core_reactions=core_reaction_ids, method='min')

2024-05-01 11:42:01,655 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Max CO2 production:  4.000003999999997
Max ATP production:  20.50002050000001
Test feasability:  0.0
Timeout limit is 3600s


2024-05-01 11:44:44,054 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-05-01 11:44:58,597 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-05-01 11:44:58,599 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-05-01 11:44:58,600 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:44:58,602 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-05-01 11:44:58,605 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-05-01 11:44:58,608 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-05-01 11:44:58,610 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:44:58,612 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-05-01 11:44:58,634 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-05-01 11:44:58,635 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min


met=co2_e: 100%|██████████| 1/1 [06:19<00:00, 379.75s/it]


In [21]:
lumps

{'EX_hdca_e': {<Metabolite co2_e at 0x1a5931e7880>: [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'h_m': 9.6875, 'nad_m': -1.9375, 'nadh_m': 1.9375, 'adp_m': 6.25, 'atp_m': -6.25, 'h2o_m': -9.125, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'fum_m': -0.5, 'o2_m': 1.4375, 'pi_m': 6.25, 'succ_m': 0.5, 'hdca_e': -0.0625, 'h_e': -0.0625, 'co2_e': 1.0, 'h2o_e': 1.0, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.5, 'ADK3': 0.06249999999999997, 'AKGDm': 0.5, 'ATPtm': 6.687499999999999, 'C160CPT1': 0.0625, 'C160CPT2': 0.0625, 'CO2tm': -1.0, 'CSm': 0.5, 'FAOXC160': 0.0625, 'FUMm': 0.5, 'H2Otm': -7.687500000000002, 'ICDHxm': 0.5, 'MDHm': 0.5, 'O2tm': 1.4375000000000007, 'SUCOAS1m': -0.06249999999999997, 'SUCOASm': -0.4375, 'r0801': -0.06249999999999997, 'r2435': 0.0625, 'PALFATPtc': 0.0625, 'The': 0.0625000000000055, 'CO2t': -1.0, 'H2Ot': -1.0000000000000013, 'O2t': 1.4375000000000007, 'PPA': 0.0625, 'cyt_atp2adp': 6.625, 'PiC': 6.750000000000005}, gene_reaction_rule='(50.1 or 4

In [22]:
# Save the lumps in a json file Lumps are NamedTuple objects
import json

lumps_json = {k: [i for m,l in v.items() for i in l ] for k,v in lumps.items()}
with open('lumps.json', 'w') as f:
    json.dump(lumps_json, f, indent=4)


In [23]:
lumps_json

{'EX_hdca_e': [Lump(id_='LUMP_EX_co2_e_co2_e', metabolites=defaultdict(<class 'int'>, {'h_m': 9.6875, 'nad_m': -1.9375, 'nadh_m': 1.9375, 'adp_m': 6.25, 'atp_m': -6.25, 'h2o_m': -9.125, 'fad_m': -0.4375, 'fadh2_m': 0.4375, 'fum_m': -0.5, 'o2_m': 1.4375, 'pi_m': 6.25, 'succ_m': 0.5, 'hdca_e': -0.0625, 'h_e': -0.0625, 'co2_e': 1.0, 'h2o_e': 1.0, 'o2_e': -1.4375}), subnetwork={'ACONTm': 0.5, 'ADK3': 0.06249999999999997, 'AKGDm': 0.5, 'ATPtm': 6.687499999999999, 'C160CPT1': 0.0625, 'C160CPT2': 0.0625, 'CO2tm': -1.0, 'CSm': 0.5, 'FAOXC160': 0.0625, 'FUMm': 0.5, 'H2Otm': -7.687500000000002, 'ICDHxm': 0.5, 'MDHm': 0.5, 'O2tm': 1.4375000000000007, 'SUCOAS1m': -0.06249999999999997, 'SUCOASm': -0.4375, 'r0801': -0.06249999999999997, 'r2435': 0.0625, 'PALFATPtc': 0.0625, 'The': 0.0625000000000055, 'CO2t': -1.0, 'H2Ot': -1.0000000000000013, 'O2t': 1.4375000000000007, 'PPA': 0.0625, 'cyt_atp2adp': 6.625, 'PiC': 6.750000000000005}, gene_reaction_rule='(50.1 or 48.1) and (26289.1 or 26289.2) and ((49

In [24]:
# Make a new model with the lumps only containing the core reactions and the lumps
reduced_model = recon3d.copy()

2024-05-01 11:52:40,340 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [25]:
boundary = [r.id for r in reduced_model.reactions if r in reduced_model.boundary]
lump_subnet_reactions = list({r for lumps in lumps_json.values() for l in lumps for r in l.subnetwork})
reduced_reactions = core_reaction_ids + lump_subnet_reactions + boundary

reactions_to_remove = [r for r in reduced_model.reactions if r.id not in reduced_reactions]
reduced_model.remove_reactions(reactions_to_remove)



In [26]:
reduced_model.prepare()
reduced_model.convert()


2024-05-01 12:05:34,803 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...
2024-05-01 12:05:49,758 - thermomodel_Recon3thermoCurated - INFO - # Model preparation done.
2024-05-01 12:05:49,759 - thermomodel_Recon3thermoCurated - INFO - # Model conversion starting...
2024-05-01 12:06:07,773 - thermomodel_Recon3thermoCurated - INFO - # Model conversion done.
2024-05-01 12:06:07,774 - thermomodel_Recon3thermoCurated - INFO - # Updating cobra_model variables...
2024-05-01 12:06:07,909 - thermomodel_Recon3thermoCurated - INFO - # cobra_model variables are up-to-date


In [29]:
# Make flux variability and remove reactions that are not active
reduced_model.objective = reduced_model.reactions.get_by_id('cyt_atp2adp') 

carbon_sources = ['EX_glc_D_e', 'EX_lac_L_e', 'EX_bhb_e', 'EX_hdca_e']

for carbon_source in carbon_sources:
    reduced_model.reactions.get_by_id(carbon_source).bounds = (-1, 0)


reduced_model.optimize()

,fluxes,reduced_costs
ACACt2m,-0.901961,None
ACONTm,12.803922,None
ADK3,1.901961,None
AKGDm,12.803922,None
ATPtm,171.392157,None
...,...,...
NADH2_u10mi,52.313725,None
CYOOm3i,0.000000,None
CYOOm2i,36.058824,None
cyt_atp2adp,171.490196,None


In [30]:
reduced_model.medium

{'EX_bhb_e': 1,
 'EX_hdca_e': 1,
 'EX_h_e': 200.0,
 'EX_h2o_e': 200.0,
 'EX_lac_L_e': 1,
 'EX_o2_e': 200.0,
 'EX_glc_D_e': 1}

In [31]:
# TVA to remove reactions that are not active
from pytfa.analysis import variability_analysis

FVA = variability_analysis(reduced_model, kind='reactions')


2024-05-01 12:17:33,901 - thermomodel_Recon3thermoCurated - INFO - Beginning variability analysis for variable of type reactions
maximizing: 100%|██████████| 1885/1885 [07:38<00:00,  4.11it/s]


In [32]:
reactions_to_remove = [r for r in reduced_model.reactions if FVA.loc[r.id, 'minimum'] >= -EPSILON and FVA.loc[r.id, 'maximum'] <= EPSILON ]
len(reactions_to_remove)

1817

In [33]:
# Load model for sake of time
# from pytfa.io.json import load_json_model
# reduced_model = load_json_model('reduced_model_no_core_20240412-090529.json')

# remove reactions that are not active min and max are zero
reactions_to_remove = [r for r in reduced_model.reactions if FVA.loc[r.id, 'minimum'] >= -EPSILON and FVA.loc[r.id, 'maximum'] <= EPSILON ]
reduced_model.remove_reactions(reactions_to_remove)
reduced_model.repair()

metabolites_to_remove = [m for m in reduced_model.metabolites if len(m.reactions) == 0]
reduced_model.remove_metabolites(metabolites_to_remove)


In [34]:
reduced_model.repair()

In [35]:
from pytfa.analysis import variability_analysis
FVA = variability_analysis(reduced_model, kind='reactions')
# Show the remaning reactions

2024-05-01 12:28:24,002 - thermomodel_Recon3thermoCurated - INFO - Beginning variability analysis for variable of type reactions
maximizing: 100%|██████████| 68/68 [00:04<00:00, 13.74it/s]


In [36]:
# Concentration ranges
from pytfa.optim.variables import LogConcentration
TVA = variability_analysis(reduced_model, kind=LogConcentration)

2024-05-01 12:28:33,649 - thermomodel_Recon3thermoCurated - INFO - Beginning variability analysis for variable of type <class 'pytfa.optim.variables.LogConcentration'>
maximizing: 100%|██████████| 76/76 [00:05<00:00, 13.05it/s]


In [37]:
from pytfa.io.json import save_json_model
import datetime 
# Save the reduced model as json
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
reduced_model.id = 'reduced_model_ETC_core_{}'.format(timestamp)

save_json_model(reduced_model, 'reduced_model_ETC_core_{}.json'.format(timestamp))


In [38]:
from cobra.io.json import save_json_model as cobra_save_json_model

cobra_save_json_model(reduced_model, 'reduced_model_ETC_core_fba_only_{}.json'.format(timestamp))

In [39]:
# from cobra.io.json import save_json_model as cobra_save_json_model

# cobra_save_json_model(recon3d, 'recon3d_fba_only.json')